# AES-128-CTR Simulation (Symmetric Encryption)

In this exercise, we will see how AES symmetric encryption works without the high-level abstraction provided by libraries such as Fernet.

### Key concepts:
1. **Symmetric Key**: Alice and Bob share the exact same secret key.
2. **AES (Advanced Encryption Standard)**: The standard block cipher algorithm.
3. **CTR Mode (Counter Mode)**: A mode that transforms AES into a stream cipher. It is ideal for IoT because it does not require padding and allows messages of any length to be encrypted.

In [ ]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
import os

def print_hex(label, data):
    print(f"{label}: {data.hex().upper()}")

## Step 1: Configuring the Key and Nonce

For AES-128, the key must be exactly 16 bytes (128 bits).
In addition, CTR mode requires a **Nonce** (a number used only once) to ensure that if we encrypt the same message twice, the output will be different.

In [ ]:
# Generate a random 16-byte key
shared_key = os.urandom(16)

# The Nonce (or initial Counter) must be 16 bytes for AES
nonce = os.urandom(16)

print_hex("Shared Key (Secret)", shared_key)
print_hex("Nonce (Pubblic)", nonce)

## Step 2: Encryption (Device A)

Device A wants to send a message. In CTR mode, the AES algorithm encrypts the *nonce* and then performs an XOR with the plaintext message.

In [ ]:
plaintext = b"Sensor Data: Temp=22.5C"

# Creating the cipher instance
cipher = Cipher(algorithms.AES(shared_key), modes.CTR(nonce))
encryptor = cipher.encryptor()

# Encryption
ciphertext = encryptor.update(plaintext) + encryptor.finalize()

print(f"Unencrypted message: {plaintext}")
print_hex("Encrypted message (travels over the network)", ciphertext)

## Step 3: Decryption (Device B)

Device B receives the `ciphertext`. To recover the original message, it must use the **same key** and the **same nonce**.

In [ ]:
# The recipient must initialize the same algorithm with the same parameters.
cipher_dec = Cipher(algorithms.AES(shared_key), modes.CTR(nonce))
decryptor = cipher_dec.decryptor()

# Deciphering
decrypted_data = decryptor.update(ciphertext) + decryptor.finalize()

print(f"Message decrypted by Bob: {decrypted_data.decode()}")

## Step 4: Vulnerability (Lack of Integrity)

AES protects confidentiality, but not integrity. Let's see what happens if an attacker changes a single bit in the encrypted message as it travels over the network.

In [ ]:
# The attacker intercepts the ciphertext and changes the last byte.
corrupted_ciphertext = bytearray(ciphertext)
corrupted_ciphertext[-1] = corrupted_ciphertext[-1] ^ 0x01 # Bit flipping attack

# Bob tries to decipher the tampered message
decryptor_corrupt = Cipher(algorithms.AES(shared_key), modes.CTR(nonce)).decryptor()
final_data = decryptor_corrupt.update(corrupted_ciphertext) + decryptor_corrupt.finalize()

print_hex("Original ciphertext ", ciphertext)
print_hex("Corrupted ciphertext ", corrupted_ciphertext)
print(f"Corrupted decryption result: {final_data}")

print("\nNOTE: Notice how Bob did not receive any error messages. He decrypted false data without knowing it.")